In [ ]:
from main import *
train_sub = [1,2,4,5,6,7,8,9]
val_sub = [3]
test_sub = [3]

train_without_aug(train_sub, val_sub, test_sub)

In [ ]:
train_sub = [1,2,3,4,6,7,8,9]
val_sub = [5]
test_sub = [5]

train_without_aug(train_sub, val_sub, test_sub)